In [2]:
# Data Processing
import pandas as pd
import numpy as np

# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

In [39]:
df = pd.read_csv("traces/upb-hyccups2012/upb2012.csv", nrows=100000)

In [40]:
df

,messageId,messageSource,messageHopCount,oldRelayId,oldFriendWithDestination,oldRelayBattery,oldCommonCommunity,oldDataMemory,newRelayId,newFriendWithDestination,newRelayBattery,newCommonCommunity,newDataMemory
0,0,0,1,5,0,0.333218,0,1.0,0,0,0.730968,0,0.0000
1,1,0,1,5,0,0.333218,0,1.0,0,0,0.730968,0,0.0000
2,2,0,1,5,0,0.333218,0,1.0,0,0,0.730968,0,0.0000
3,3,0,1,5,0,0.333218,0,1.0,0,0,0.730968,0,0.0000
4,4,0,1,5,0,0.333218,0,1.0,0,0,0.730968,0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1795,2,3,0,0,0.730968,0,1.0,4,0,0.597545,0,0.0704
99996,1796,2,0,0,0,0.730968,1,1.0,4,0,0.597545,1,0.0704
99997,1797,2,0,0,0,0.730968,1,1.0,4,0,0.597545,1,0.0704
99998,1798,2,0,0,0,0.730968,1,1.0,4,0,0.597545,1,0.0704


In [41]:
successful_messages = pd.read_csv("traces/upb-hyccups2012/successful2012.csv", nrows=10000)

In [42]:
successful_messages

,messageId,oldId,destId
0,14,5,0
1,579,5,0
2,595,5,0
3,604,6,1
4,1176,6,1
...,...,...,...
9995,1797,4,0
9996,1798,4,0
9997,1291,4,0
9998,1864,4,0


In [43]:
df["usefulTransfer"] = np.NaN
df
df[df['messageId']==120]

,messageId,messageSource,messageHopCount,oldRelayId,oldFriendWithDestination,oldRelayBattery,oldCommonCommunity,oldDataMemory,newRelayId,newFriendWithDestination,newRelayBattery,newCommonCommunity,newDataMemory,usefulTransfer
1329,120,5,0,0,0,0.730968,1,1.0,2,1,0.637417,0,0.0001,NaN
1484,120,5,0,0,0,0.730968,1,1.0,2,1,0.637417,0,0.0109,NaN
1711,120,5,0,0,0,0.730968,1,1.0,2,1,0.637417,0,0.0109,NaN
1938,120,5,0,0,0,0.730968,1,1.0,2,1,0.637417,0,0.0109,NaN
2165,120,5,0,0,0,0.730968,1,1.0,2,1,0.637417,0,0.0109,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95251,120,5,0,0,0,0.730968,1,1.0,4,0,0.597545,0,0.0704,NaN
97119,120,5,0,2,1,0.637417,1,1.0,4,0,0.597545,1,0.0425,NaN
97502,120,5,0,0,0,0.730968,1,1.0,2,1,0.637417,1,0.0702,NaN
99157,120,5,0,0,0,0.730968,1,1.0,4,0,0.597545,1,0.0425,NaN


In [44]:
for index, row in successful_messages.iterrows():
  message_id = row.iloc[0]
  last_relay = row.iloc[1]
  destination = row.iloc[2]

  reached_source = False
  queue = df[(df['messageId'] == message_id) & (df['newRelayId'] == last_relay)].index.tolist()
  visited = []
  while (True):
    if (len(queue) == 0):
       break

    curr_idx = queue.pop()
    visited.append(curr_idx)

    df_row = df.iloc[curr_idx]
    message_source = df_row.iloc[1]
    old_relay_id = df_row.iloc[3]

    if old_relay_id == message_source:
        break

    df.loc[curr_idx, "usefulTransfer"] = 1
    last_relay = old_relay_id


    for idx in df[(df['messageId'] == message_id) & (df['newRelayId'] == last_relay)].index.tolist():
       if (idx not in visited):
         queue.insert(0, idx)

df.loc[(df['usefulTransfer'] != 1), 'usefulTransfer'] = 0

print(df['usefulTransfer'].value_counts()[1])

df.to_csv("traces/upb-hyccups2012/with_result.csv")


8385
